In [2]:
import os
from dbr import *

def config_accuracy_first(dbr:BarcodeReader):

    # Obtain current runtime settings of instance.
    sts = dbr.get_runtime_settings()
    
    # 1. Set expected barcode format
    # The more precise the barcode format is set, the higher the accuracy.
    # Mostly, misreading only occurs on reading oneD barcode. So here we use OneD barcode format to demonstrate.
    sts.barcode_format_ids = EnumBarcodeFormat.BF_ONED
    sts.barcode_format_ids_2 = EnumBarcodeFormat.BF_NULL
    
    # 2. Set the minimal result confidence.
    # The greater the confidence, the higher the accuracy.
    # Filter by minimal confidence of the decoded barcode. We recommend using 30 as the default minimal confidence value
    sts.minResultConfidence = 30
    
    # 3. Sets the minimum length of barcode text for filtering.
    # The more precise the barcode text length is set, the higher the accuracy.
    sts.minBarcodeTextLength = 6
    
    # Apply the new settings to the instance
    dbr.update_runtime_settings(sts)

def config_accuracy_first_by_template(dbr:BarcodeReader):
    # Compared with PublicRuntimeSettings, parameter templates have a richer ability to control parameter details.
    # Please refer to the parameter explanation in "accuracy-first-template.json" to understand how to control accuracy first.
    template_path = "/Users/xbkaishui/opensource/cv_hz/cv_tools_box/chores/accuracy-first-template.json"
    error = dbr.init_runtime_settings_with_file(template_path, EnumConflictMode.CM_OVERWRITE)
    if error[0] != EnumErrorCode.DBR_OK:
        print("init_runtime_settings_with_file error: " + error[1])

def output_results(results:TextResult):
    if results != None:
        i = 0
        for res in results:
            barcode_format = res.barcode_format_string

            print("Barcode " + str(i) + ":" + barcode_format + "," + res.barcode_text)
            i = i+1
    else:
        print("No data detected.")


if __name__ == "__main__":
    try:
        # 1.Initialize license.
        # The string "DLS2eyJvcmdhbml6YXRpb25JRCI6IjIwMDAwMSJ9" here is a free public trial license. Note that network connection is required for this license to work.
        # You can also request a 30-day trial license in the customer portal: https://www.dynamsoft.com/customer/license/trialLicense?architecture=dcv&product=dbr&utm_source=samples&package=python
        error = BarcodeReader.init_license("DLS2eyJvcmdhbml6YXRpb25JRCI6IjIwMDAwMSJ9")
        if error[0] != EnumErrorCode.DBR_OK:
            print("License error: "+ error[1])

        # 2.Create an instance of Barcode Reader.
        dbr = BarcodeReader.get_instance()
        if dbr == None:
            raise BarcodeReaderError("Get instance failed")
        # Replace by your own image path
        image_path = "/tmp/contours_image.png"

        # Accuracy = The number of correctly decoded barcodes/the number of all decoded barcodes
        # There are two ways to configure runtime parameters. One is through PublicRuntimeSettings, the other is through parameters template.
        print("Decode through PublicRuntimeSettings:")

        # 3.a config through PublicRuntimeSettings
        config_accuracy_first(dbr)
        
        # 4.a Decode barcodes from an image file by current runtime settings. The second parameter value "" means to decode through the current PublicRuntimeSettings.
        results = dbr.decode_file(image_path)
        
        # 5.a Output the barcode format and barcode text.
        output_results(results)
        
        print("\nDecode through parameters template:")
        # 3.b config through parameters template
        config_accuracy_first_by_template(dbr)
        
        # 4.b Decode barcodes from an image file by template. 
        results = dbr.decode_file(image_path, "")
        
        # 5.b Output the barcode format and barcode text.
        output_results(results)
        
        dbr.recycle_instance()
    except BarcodeReaderError as bre:
        print(bre)

License error: DLS Connect Error.-ErrorCode:7-HttpCode:0-ReponseStr:
Decode through PublicRuntimeSettings:
No data detected.

Decode through parameters template:
No data detected.


In [9]:
import os
import sys
from dbr import *

def output_results(results:TextResult):
    if results != None:
        i = 0
        for res in results:
            print(res)
            barcode_format = res.barcode_format_string

            print("Barcode " + str(i) + ":" + barcode_format + "," + res.barcode_text)
            i = i+1
    else:
        print("No data detected.")

try:
    # 1.Initialize license.
    # The string "DLS2eyJvcmdhbml6YXRpb25JRCI6IjIwMDAwMSJ9" here is a free public trial license. Note that network connection is required for this license to work.
    # You can also request a 30-day trial license in the customer portal: https://www.dynamsoft.com/customer/license/trialLicense?architecture=dcv&product=dbr&utm_source=samples&package=python
    error = BarcodeReader.init_license("t0069lQAAACey/VlrVhOw6DL2qoMvvGZB1vNTixAke6sUHXVNBSKM3aYQslybwo675KtCiOHktJDXQB5wg/7NlP3fz3W8rMY1")
    if error[0] != EnumErrorCode.DBR_OK:
        print("License error: "+ error[1])

    # 2.Create an instance of Barcode Reader.
    dbr = BarcodeReader.get_instance()
    if dbr == None:
        raise BarcodeReaderError("Get instance failed")
    # 3.Config runtime settings for dmp
    sts = dbr.get_runtime_settings()
    
    # 3.1. Set expected barcode formats. 
    # Generally, the most common dpm barcode is datamatrix or qrcode
    sts.barcode_format_ids = EnumBarcodeFormat.BF_DATAMATRIX | EnumBarcodeFormat.BF_QR_CODE

    # 3.2. Set grayscale transformation modes.
    # DPM barcodes may be dark and printed on the light background, or it may be light and printed on the dark background.
    # By default, the library can only locate the dark barcodes that stand on a light background. "GTM_INVERTED":The image will be transformed into inverted grayscale.
    sts.grayscale_transformation_modes = [EnumGrayscaleTransformationMode.GTM_ORIGINAL,EnumGrayscaleTransformationMode.GTM_INVERTED,0,0,0,0,0,0]

    # 3.3. Enable dpm modes.
    # It is a parameter to control how to read direct part mark (DPM) barcodes.
    sts.dpm_code_reading_modes = [EnumDPMCodeReadingMode.DPMCRM_GENERAL,0,0,0,0,0,0,0]

    dbr.update_runtime_settings(sts)

    # Replace with your own dpm barcode image path
    # image_path = os.path.dirname(os.path.abspath(__file__)) + os.path.sep + "../../images/dpm.jpg"
    image_path = "/tmp/contours_image.png"
    image_path = "/Users/xbkaishui/opensource/cv_hz/cv_tools_box/cv2/test.bmp"
    image_path = "/tmp/test2.bmp"
    # image_path = "/tmp/test.png"

    # 3 Decode barcodes from an image file by current runtime settings. The second parameter value "" means to decode through the current PublicRuntimeSettings.
    results = dbr.decode_file(image_path,"")
    
    # 4 Output the barcode format and barcode text.
    output_results(results)
    
    dbr.recycle_instance()
    
except BarcodeReaderError as bre:
    print(bre)

Barcode 0:DATAMATRIX,#8BG505235A###011223=050A


In [11]:
import json 
dbr = BarcodeReader.get_instance()
sts = dbr.get_runtime_settings()

sts
# json.dumps(sts)